In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


Calculate Ecommerce Metrics By Date

In [ ]:
%%bigquery --project produkcja-instore

select 
    event_date as date,
    count(distinct ecommerce.transaction_id) as transactions,
    sum(ecommerce.purchase_revenue) as purchase_revenue
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between '20210701' and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by 
    date
order by
    date

LIMIT 4

,date,transactions,purchase_revenue
0,20210701,1344,64179.0
1,20210702,1051,50756.0
2,20210703,233,11683.0
3,20210704,218,10344.0


Calculate Conversion Rate By Engaged Sessions

In [ ]:
%%bigquery --project produkcja-instore

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    event_date,
    max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged,
    ecommerce.transaction_id,
    ecommerce.purchase_revenue
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between '20210701' and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by
    user_pseudo_id,
    session_id,
    event_date,
    transaction_id,
    purchase_revenue)

select
    event_date as date,
    count(distinct transaction_id) as transactions,
    sum(purchase_revenue) as purchase_revenue,
    count(distinct transaction_id) / count(distinct concat(user_pseudo_id,session_id)) as ecommerce_conversion_rate_all_sessions,
    count(distinct transaction_id) / count(distinct case when session_engaged = '1' then concat(user_pseudo_id,session_id) else null end) as ecommerce_conversion_rate_engaged_sessions
from
    prep
group by 
    date
order by 
    date

LIMIT 4

,date,transactions,purchase_revenue,ecommerce_conversion_rate_all_sessions,ecommerce_conversion_rate_engaged_sessions
0,20210701,1344,64179.0,0.225428,0.370452
1,20210702,1051,50756.0,0.231804,0.384559
2,20210703,233,11683.0,0.220853,0.393581
3,20210704,218,10344.0,0.201293,0.352181


Add Refund, Tax, And Shipping Data

In [ ]:
%%bigquery --project produkcja-instore

with prep as (
select
    user_pseudo_id,
    (select value.int_value from unnest(event_params) where key = 'ga_session_id') as session_id,
    event_date,
    max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged,
    ecommerce.transaction_id,
    ecommerce.purchase_revenue,
    ecommerce.total_item_quantity,
    ecommerce.refund_value,
    ecommerce.tax_value,
    ecommerce.shipping_value
from
    `simoahava-com.analytics_206575074.events_*`
where
    _table_suffix between '20210701' and format_date('%Y%m%d',date_sub(current_date(), interval 1 day))
group by
    user_pseudo_id,
    session_id,
    event_date,
    transaction_id,
    purchase_revenue,
    total_item_quantity,
    refund_value,
    tax_value,
    shipping_value)

select
    event_date as date,
    count(distinct transaction_id) as transactions,
    sum(purchase_revenue) as purchase_revenue,
    count(distinct transaction_id) / count(distinct concat(user_pseudo_id,session_id)) as ecommerce_conversion_rate_all_sessions,
    count(distinct transaction_id) / count(distinct case when session_engaged = '1' then concat(user_pseudo_id,session_id) else null end) as ecommerce_conversion_rate_engaged_sessions,
    sum(total_item_quantity) as items,
    sum(refund_value) as refund_value,
    sum(tax_value) as tax_value,
    sum(shipping_value) as shipping_value,
    safe_divide(sum(purchase_revenue),count(distinct transaction_id)) as average_transaction_value,
    safe_divide(sum(shipping_value),count(distinct transaction_id)) as average_shipping_value,
    safe_divide(sum(total_item_quantity),count(distinct transaction_id)) as average_items
from
    prep
group by 
    date
order by 
    date
LIMIT 4

,date,transactions,purchase_revenue,ecommerce_conversion_rate_all_sessions,ecommerce_conversion_rate_engaged_sessions,items,refund_value,tax_value,shipping_value,average_transaction_value,average_shipping_value,average_items
0,20210701,1344,64179.0,0.225428,0.370452,13093,None,12421.91,5738.5,47.752232,4.269717,9.741815
1,20210702,1051,50756.0,0.231804,0.384559,9584,None,9824.07,4477.5,48.293054,4.260228,9.118934
2,20210703,233,11683.0,0.220853,0.393581,2406,None,2261.25,965.0,50.141631,4.141631,10.326180
3,20210704,218,10344.0,0.201293,0.352181,2715,None,2002.22,920.0,47.449541,4.220183,12.454128
